In [1]:
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Dense,Input,Activation,Flatten
from tensorflow.keras.regularizers import l1,l2
from tensorflow.keras.models import Model
import tensorflow as tf
import numpy as np

filters_per_conv_layer = [16,16]
neurons_per_dense_layer = [32,64]
batch_size=250

x = x_in = Input((7,7,1))

for i,f in enumerate(filters_per_conv_layer):
    print( ('Adding convolutional block {} with N={} filters').format(i,f) )
    x = Conv2D(int(f), kernel_size=(2,2), strides=(1,1), kernel_initializer='lecun_uniform',padding="same", kernel_regularizer=l1(0.0001), use_bias=False,
               name='conv_{}'.format(i))(x) 
    x = BatchNormalization(name='bn_conv_{}'.format(i))(x)
    x = Activation('relu',name='conv_act_%i'%i)(x)


x = MaxPooling2D(pool_size = (2,2))(x)

x = Flatten()(x)

for i,n in enumerate(neurons_per_dense_layer):
    print( ('Adding dense block {} with N={} neurons').format(i,n) )
    x = Dense(n,kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001),name='dense_%i'%i, use_bias=False)(x)
    x = BatchNormalization(name='bn_dense_{}'.format(i))(x)
    x = Activation('relu',name='dense_act_%i'%i)(x)
x = Dense(2,name='output_dense')(x)
x_out = Activation('softmax',name='output_softmax')(x)

model = Model(inputs=[x_in], outputs=[x_out], name='keras_baseline')
model.compile(loss='categorical_crossentropy',
                      optimizer=tf.keras.optimizers.Adam(learning_rate=1.578e-5),
                      metrics=['accuracy'],)


2023-02-27 10:59:52.490037: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-27 10:59:52.490053: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Adding convolutional block 0 with N=16 filters
Adding convolutional block 1 with N=16 filters
Adding dense block 0 with N=32 neurons
Adding dense block 1 with N=64 neurons


2023-02-27 10:59:53.557268: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-27 10:59:53.557288: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-27 10:59:53.557299: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ej-ubt): /proc/driver/nvidia/version does not exist
2023-02-27 10:59:53.557465: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
model.summary()

Model: "keras_baseline"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 7, 7, 1)]         0         
                                                                 
 conv_0 (Conv2D)             (None, 7, 7, 16)          64        
                                                                 
 bn_conv_0 (BatchNormalizati  (None, 7, 7, 16)         64        
 on)                                                             
                                                                 
 conv_act_0 (Activation)     (None, 7, 7, 16)          0         
                                                                 
 conv_1 (Conv2D)             (None, 7, 7, 16)          1024      
                                                                 
 bn_conv_1 (BatchNormalizati  (None, 7, 7, 16)         64        
 on)                                                

## generate CNN model io stream, non-quantized

In [3]:
from tensorflow_model_optimization.sparsity.keras import strip_pruning
model  = strip_pruning(model)

In [4]:
import hls4ml
import plotting
import os
os.environ['PATH'] = '/opt/Xilinx/Vivado/2019.2/bin:' + os.environ['PATH']

hls_config = hls4ml.utils.config_from_keras_model(model, granularity='name')

# Set the precision and reuse factor for the full model
hls_config['Model']['Precision'] = 'ap_fixed<16,6>'
hls_config['Model']['ReuseFactor'] = 1

# Create an entry for each layer, here you can for instance change the strategy for a layer to 'resource' 
# or increase the reuse factor individually for large layers.
# In this case, we designed the model to be small enough for a fully parallel implementation 
# so we use the latency strategy and reuse factor of 1 for all layers.
for Layer in hls_config['LayerName'].keys():
    hls_config['LayerName'][Layer]['Strategy'] = 'Resource'
    hls_config['LayerName'][Layer]['ReuseFactor'] = 2
#If you want best numerical performance for high-accuray models, while the default latency strategy is faster but numerically more unstable
hls_config['LayerName']['output_softmax']['Strategy'] = 'Stable'

cfg = hls4ml.converters.create_config(backend='Vivado')
cfg['IOType']     = 'io_stream'
cfg['HLSConfig']  = hls_config
cfg['KerasModel'] = model
cfg['OutputDir']  = 'cnn_io_stream_resource_7x7'
cfg['Part'] = 'xcvu9p-flga2104-2L-e'
  
hls_model = hls4ml.converters.keras_to_hls(cfg)
hls_model.compile()

Interpreting Model
Topology:
Layer name: input_1, layer type: Input
Layer name: conv_0, layer type: Conv2D
  -> Activation (linear), layer name: conv_0
Layer name: bn_conv_0, layer type: BatchNormalization
Layer name: conv_act_0, layer type: Activation
Layer name: conv_1, layer type: Conv2D
  -> Activation (linear), layer name: conv_1
Layer name: bn_conv_1, layer type: BatchNormalization
Layer name: conv_act_1, layer type: Activation
Layer name: max_pooling2d, layer type: MaxPooling2D
Layer name: dense_0, layer type: Dense
  -> Activation (linear), layer name: dense_0
Layer name: bn_dense_0, layer type: BatchNormalization
Layer name: dense_act_0, layer type: Activation
Layer name: dense_1, layer type: Dense
  -> Activation (linear), layer name: dense_1
Layer name: bn_dense_1, layer type: BatchNormalization
Layer name: dense_act_1, layer type: Activation
Layer name: output_dense, layer type: Dense
  -> Activation (linear), layer name: output_dense
Layer name: output_softmax, layer type:

/home/ej/workspace/hls4ml/hls4ml/hls4ml/converters/__init__.py:15: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!")


Done


In [5]:
hls_model.build(csim=False, synth=True, vsynth=True)


****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2019.2 (64-bit)
  **** SW Build 2708876 on Wed Nov  6 21:39:14 MST 2019
  **** IP Build 2700528 on Thu Nov  7 00:09:20 MST 2019
    ** Copyright 1986-2019 Xilinx, Inc. All Rights Reserved.

source /opt/Xilinx/Vivado/2019.2/scripts/vivado_hls/hls.tcl -notrace
INFO: [HLS 200-10] Running '/opt/Xilinx/Vivado/2019.2/bin/unwrapped/lnx64.o/vivado_hls'
INFO: [HLS 200-10] For user 'ej' on host 'ej-ubt' (Linux_x86_64 version 5.4.0-139-generic) on Mon Feb 27 10:59:58 PST 2023
INFO: [HLS 200-10] On os Ubuntu 18.04.6 LTS
INFO: [HLS 200-10] In directory '/home/ej/workspace/gluon_tagger_algo/cnn_io_stream_resource_7x7'
Sourcing Tcl script 'build_prj.tcl'
INFO: [HLS 200-10] Creating and opening project '/home/ej/workspace/gluon_tagger_algo/cnn_io_stream_resource_7x7/myproject_prj'.
INFO: [HLS 200-10] Adding design file 'firmware/myproject.cpp' to the project
INFO: [HLS 200-10] Adding test bench file 'myproject_test.cpp' to the proj

INFO: [XFORM 203-602] Inlining function 'nnet::cast<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config2_mult>' into 'nnet::conv_2d_cl<nnet::array<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 1u>, nnet::array<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 16u>, config2>' (firmware/nnet_utils/nnet_dense_resource.h:100->firmware/nnet_utils/nnet_dense_resource.h:276->firmware/nnet_utils/nnet_conv_stream.h:279->firmware/nnet_utils/nnet_conv2d_stream.h:85->firmware/nnet_utils/nnet_conv2d_stream.h:103) automatically.
INFO: [XFORM 203-602] Inlining function 'nnet::cast<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config6_mult>' into 'nnet::conv_2d_cl<nnet::array<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 16u>, nnet::array<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 16u>, config6>' (firmware/nnet_utils/nnet_dense_resource.h:100->firmware/nnet_utils/nnet_dense_resource.h:276->firmw

INFO: [HLS 200-489] Unrolling loop 'ReLUPackLoop' (firmware/nnet_utils/nnet_activation_stream.h:67) in function 'nnet::relu<nnet::array<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 32u>, nnet::array<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 32u>, relu_config15>' completely with a factor of 32.
INFO: [HLS 200-489] Unrolling loop 'ReLUPackLoop' (firmware/nnet_utils/nnet_activation_stream.h:67) in function 'nnet::relu<nnet::array<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 16u>, nnet::array<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 16u>, relu_config9>' completely with a factor of 16.
INFO: [HLS 200-489] Unrolling loop 'ReLUPackLoop' (firmware/nnet_utils/nnet_activation_stream.h:67) in function 'nnet::relu<nnet::array<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 16u>, nnet::array<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 16u>, relu_config5>' completely with a factor of 16.
INFO: [HLS 200-489] Unrolling loop 'FiltLoop' (firmware/nnet_utils/nnet_pooling_stream.h:1

INFO: [XFORM 203-102] Partitioning array 'res_part.data.V' (firmware/nnet_utils/nnet_padding_stream.h:23) automatically.
INFO: [XFORM 203-102] Partitioning array 'exp_res.V' (firmware/nnet_utils/nnet_activation_stream.h:270) automatically.
INFO: [XFORM 203-102] Partitioning array 'data_cache.V' (firmware/nnet_utils/nnet_activation_stream.h:272) automatically.
INFO: [XFORM 203-131] Reshaping array 'w2.V'  in dimension 1 with a block factor of 32.
INFO: [XFORM 203-131] Reshaping array 'w6.V'  in dimension 1 with a block factor of 512.
INFO: [XFORM 203-131] Reshaping array 'w20.V'  in dimension 1 with a block factor of 64.
INFO: [XFORM 203-131] Reshaping array 'w16.V'  in dimension 1 with a block factor of 1024.
INFO: [XFORM 203-131] Reshaping array 'w12.V'  in dimension 1 with a block factor of 2304.
INFO: [XFORM 203-102] Automatically partitioning shift register array 'line_buffer.Array.V.2' .
INFO: [XFORM 203-102] Automatically partitioning shift register array 'line_buffer.Array.V.1' 

INFO: [XFORM 203-602] Inlining function 'nnet::cast<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config2_mult>' into 'nnet::conv_2d_cl<nnet::array<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 1u>, nnet::array<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 16u>, config2>' (firmware/nnet_utils/nnet_dense_resource.h:100->firmware/nnet_utils/nnet_dense_resource.h:276->firmware/nnet_utils/nnet_conv_stream.h:279->firmware/nnet_utils/nnet_conv2d_stream.h:85->firmware/nnet_utils/nnet_conv2d_stream.h:103) automatically.
INFO: [XFORM 203-602] Inlining function 'nnet::cast<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config6_mult>' into 'nnet::conv_2d_cl<nnet::array<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 16u>, nnet::array<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, 16u>, config6>' (firmware/nnet_utils/nnet_dense_resource.h:100->firmware/nnet_utils/nnet_dense_resource.h:276->firmw

...

INFO: [RTGEN 206-104] Estimated max fanout for 'conv_2d_cl_array_array_ap_fixed_16u_config6_s' is 8176 from HDL expression: ((1'b1 == ap_CS_fsm_pp0_stage0) & (ap_enable_reg_pp0_iter1 == 1'b1) & (1'b0 == ap_block_pp0_stage0))
INFO: [RTGEN 206-100] Generating core module 'myproject_mul_mul_16s_16s_26_1_1': 511 instance(s).
INFO: [RTGEN 206-100] Generating core module 'myproject_mul_mul_9s_16s_25_1_1': 1 instance(s).
INFO: [RTGEN 206-100] Finished creating RTL model for 'conv_2d_cl_array_array_ap_fixed_16u_config6_s'.
INFO: [HLS 200-111]  Elapsed time: 0.96 seconds; current allocated memory: 990.083 MB.
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [HLS 200-10] -- Generating RTL for module 'relu_array_array_ap_fixed_16u_relu_config9_s' 
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [RTGEN 206-100] Finished creating RTL model for 'relu_array_array_ap_fixed_16u_relu_config9_s'.
INFO: [HLS 200-111]  Elaps

INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [HLS 200-10] -- Generating RTL for module 'dense_wrapper_ap_fixed_ap_fixed_16_6_5_3_0_config12_s' 
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [RTGEN 206-104] Estimated max fanout for 'dense_wrapper_ap_fixed_ap_fixed_16_6_5_3_0_config12_s' is 36855 from HDL expression: ((1'b0 == ap_block_pp0_stage0) & (ap_enable_reg_pp0_iter1 == 1'b1) & (1'b1 == ap_CS_fsm_pp0_stage0))
INFO: [RTGEN 206-100] Generating core module 'myproject_mul_mul_16s_16s_26_1_1': 2303 instance(s).
INFO: [RTGEN 206-100] Generating core module 'myproject_mul_mul_7ns_16s_23_1_1': 1 instance(s).
INFO: [RTGEN 206-100] Finished creating RTL model for 'dense_wrapper_ap_fixed_ap_fixed_16_6_5_3_0_config12_s'.
INFO: [HLS 200-111]  Elapsed time: 9.44 seconds; current allocated memory: 1.182 GB.
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [HLS 200-10]

INFO: [RTMG 210-279] Implementing memory 'dense_wrapper_ap_fixed_ap_fixed_16_6_5_3_0_config16_s_w16_V_rom' using auto ROMs.
INFO: [RTMG 210-279] Implementing memory 'dense_wrapper_ap_fixed_ap_fixed_16_6_5_3_0_config20_s_w20_V_rom' using auto ROMs.
INFO: [RTMG 210-279] Implementing memory 'softmax_legacy_array_array_ap_fixed_2u_softmax_config22_sIfE_rom' using auto ROMs.
INFO: [RTMG 210-279] Implementing memory 'softmax_legacy_array_array_ap_fixed_2u_softmax_config22_sJfO_rom' using auto ROMs.
INFO: [RTMG 210-285] Implementing FIFO 'layer23_out_V_data_0_V_U(fifo_w16_d64_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer2_out_V_data_0_V_U(fifo_w16_d49_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer2_out_V_data_1_V_U(fifo_w16_d49_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer2_out_V_data_2_V_U(fifo_w16_d49_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer2_out_V_data_3_V_U(fifo_w16_d49_A)' usin

INFO: [RTMG 210-285] Implementing FIFO 'layer10_out_V_data_1_V_U(fifo_w16_d9_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer10_out_V_data_2_V_U(fifo_w16_d9_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer10_out_V_data_3_V_U(fifo_w16_d9_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer10_out_V_data_4_V_U(fifo_w16_d9_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer10_out_V_data_5_V_U(fifo_w16_d9_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer10_out_V_data_6_V_U(fifo_w16_d9_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer10_out_V_data_7_V_U(fifo_w16_d9_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer10_out_V_data_8_V_U(fifo_w16_d9_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer10_out_V_data_9_V_U(fifo_w16_d9_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'layer10_out_V_data_10_V_

{'CSynthesisReport': {'EstimatedClockPeriod': '4.366',
  'BestLatency': '395',
  'WorstLatency': '396',
  'IntervalMin': '129',
  'IntervalMax': '385',
  'BRAM_18K': '1819',
  'DSP48E': '3936',
  'FF': '67229',
  'LUT': '100319',
  'URAM': '0',
  'AvailableBRAM_18K': '4320',
  'AvailableDSP48E': '6840',
  'AvailableFF': '2364480',
  'AvailableLUT': '1182240',
  'AvailableURAM': '960'}}

In [6]:
hls4ml.report.read_vivado_report('cnn_io_stream_resource_7x7')

Found 1 solution(s) in cnn_io_stream_resource_7x7/myproject_prj.
Reports for solution "solution1":

C simulation report not found.
SYNTHESIS REPORT:
== Vivado HLS Report for 'myproject'
* Date:           Mon Feb 27 11:09:38 2023

* Version:        2019.2 (Build 2704478 on Wed Nov 06 22:10:23 MST 2019)
* Project:        myproject_prj
* Solution:       solution1
* Product family: virtexuplus
* Target device:  xcvu9p-flga2104-2L-e


== Performance Estimates
+ Timing: 
    * Summary: 
    +--------+---------+----------+------------+
    |  Clock |  Target | Estimated| Uncertainty|
    +--------+---------+----------+------------+
    |ap_clk  | 5.00 ns | 4.366 ns |   0.62 ns  |
    +--------+---------+----------+------------+

+ Latency: 
    * Summary: 
    +---------+---------+----------+----------+-----+-----+----------+
    |  Latency (cycles) |  Latency (absolute) |  Interval | Pipeline |
    |   min   |   max   |    min   |    max   | min | max |   Type   |
    +---------+---------+--